In [ ]:
%load_ext watermark
%watermark  -d -u -a 'Andreas Mueller, Kyle Kastner, Sebastian Raschka' -v -p numpy,scipy,matplotlib,pillow,scikit-learn

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# 监督学习(Supervised Learning) Part 1 -- 分类(Classification)

为了将机器学习算法的机制可视化，研究二维或一维数据(即只有一个或两个特征的数据)通常是有帮助的。虽然在实践中，数据集通常有更多维的特征，但很难在二维屏幕上绘制高维数据。

在进入更“真实世界”的数据集之前，先来看一些非常简单的例子。

首先，看一个两维的两类分类问题，使用由``make_blobs``函数生成的数据。

In [ ]:
from sklearn.datasets import make_blobs

X, y = make_blobs(centers=2, random_state=0)

print('X ~ n_samples x n_features:', X.shape)
print('y ~ n_samples:', y.shape)

print('\nFirst 5 samples:\n', X[:5, :])
print('\nFirst 5 labels:', y[:5])

由于数据是二维的，我们可以将每个样本绘制为二维坐标系中的一个点，第一个特征对应x轴，第二个特征对应y轴。

In [ ]:
plt.scatter(X[y == 0, 0], X[y == 0, 1], 
            c='blue', s=40, label='0')
plt.scatter(X[y == 1, 0], X[y == 1, 1], 
            c='red', s=40, label='1', marker='s')

plt.xlabel('first feature')
plt.ylabel('second feature')
plt.legend(loc='upper right');

分类是一项有监督任务，我们对它在没见过的数据上的表现感兴趣，将数据分成两部分:

1. 学习算法用来拟合模型的训练集 
2. 用来评价模型泛化性能的测试集

用``model_selection``模块下的``train_test_split``函数，将数据集分割成75%的训练数据和25%的测试数据。

<img src="figures/train_test_split_matrix.svg" width="60%">


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.25,
                                                    random_state=1234,
                                                    stratify=y)

### The scikit-learn estimator API
<img src="figures/supervised_workflow.svg" width="60%">


每种算法都在scikit-learn中以一个“估计器(Estimator)”对象提供。(scikit-learn中所有模型都有一个非常一致的界面)。例如，我们首先导入逻辑回归类。

In [ ]:
from sklearn.linear_model import LogisticRegression

接下来，实例化估计器对象

In [ ]:
classifier = LogisticRegression()

In [ ]:
X_train.shape

In [ ]:
y_train.shape

要为数据建立模型，也就是学习如何对新的点进行分类，我们用训练数据和相应的训练标签(训练数据点的期望输出)调用``fit``函数:

In [ ]:
classifier.fit(X_train, y_train)

(类似`fit`的一些估计器方法会默认返回`self`，因此，在执行了上面的代码片段后，会看到这个`LogisticRegression`特定实例的默认参数。另一种获取估计器初始化参数的方法是执行`classifier.get_params()`，会返回一个参数字典)

然后，可以将该模型应用于没见过的数据，用``predict``方法使用该模型来预测估计结果:

In [ ]:
prediction = classifier.predict(X_test)

将预测结果与真正的标签做比较:

In [ ]:
print(prediction)
print(y_test)

可以通过测量预测正确结果所占比例来定量评价分类器，这叫做**准确度(accuracy)**:

In [ ]:
np.mean(prediction == y_test)

还有一个方便的函数：``score``, 所有scikit-learn分类器都必须直接用测试数据直接计算得出准确度:

In [ ]:
classifier.score(X_test, y_test)

将(测试集上的)泛化性能与训练集上的性能进行比较通常会很有帮助:

In [ ]:
classifier.score(X_train, y_train)

LogisticRegression是所谓的线性模型，意味着它将在输入空间创建一个线性决策。在2D空间，意味着它会找到一条线来区分蓝色和红色:

In [ ]:
from figures import plot_2d_separator

plt.scatter(X[y == 0, 0], X[y == 0, 1], 
            c='blue', s=40, label='0')
plt.scatter(X[y == 1, 0], X[y == 1, 1], 
            c='red', s=40, label='1', marker='s')

plt.xlabel("first feature")
plt.ylabel("second feature")
plot_2d_separator(classifier, X)
plt.legend(loc='upper right')

**估计参数(Estimated parameters)**:所有估计的模型参数，都是估计器对象的以下划线结尾的属性。这里是系数和线的偏移量:

In [ ]:
print(classifier.coef_)
print(classifier.intercept_)

另一个分类器: K近邻(K Nearest Neighbors)
------------------------------------------------
另一个流行且易于理解的分类器是K近邻分类器。它有一个非常简单的学习策略:给定一个新的、未知的观察，在参考数据库中找到那些具有最接近特征的样本，并将优势类指定为结果。

其接口与上面的``LogisticRegression above``估计器完全相同。

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

设置一个KNeighborsClassifier参数，告诉它我们只想参考一个最近邻:

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1)

我们用训练数据拟合模型

In [ ]:
knn.fit(X_train, y_train)

In [ ]:
plt.scatter(X[y == 0, 0], X[y == 0, 1], 
            c='blue', s=40, label='0')
plt.scatter(X[y == 1, 0], X[y == 1, 1], 
            c='red', s=40, label='1', marker='s')

plt.xlabel("first feature")
plt.ylabel("second feature")
plot_2d_separator(knn, X)
plt.legend(loc='upper right');

In [ ]:
knn.score(X_test, y_test)

练习
=========
将KNeighborsClassifier应用于``iris``数据集。给``n_neighbors``设置不同的值，观察训练集和测试集得分的变化。

In [ ]:
# %load solutions/05A_knn_with_diff_k.py